In [12]:
import json
from IPython.display import display, Math
import os
import pandas as pd
import numpy as np
import prettytable as pt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import dates
import seaborn as sns
from scipy.stats import norm
import datetime
import warnings
warnings.filterwarnings("ignore")


###################################################################################################
                                                                                                  #
from rhoova.Client import *                                                                       #   
#Register and get api key from https://app.rhoova.com/ for ClientConfig("api key", "api secret")  #
config = ClientConfig("", "")
api = Api(config)                                                                                 # 
                                                                                                  #
###################################################################################################

directory = os.path.normpath(os.getcwd() + os.sep )
datadirectory=directory+"/data/yielddata/yielddata.csv"


yielddata = pd.read_csv(datadirectory)
yielddata = yielddata.replace(np.nan, '', regex=True) 

mdirectory = os.path.normpath(os.getcwd() + os.sep)
mdatadirectory=mdirectory+"/data/marketdata/marketdata.csv"

marketdata = pd.read_csv(mdatadirectory)
marketdata = marketdata.replace(np.nan, '', regex=True) 

vdirectory = os.path.normpath(os.getcwd() + os.sep )
vdatadirectory=directory+"/data/volatilitydata/swaptionvol.csv"

voldata = pd.read_csv(vdatadirectory)
voldata = voldata.replace(np.nan, '', regex=True) 


In [13]:
def get_result(riskdata):
    try:
        res = api.createTask(CalculationType.PORTFOLIO, riskdata, True)
        if(res["result"]):
          result=json.loads(res["result"])
        else:
          print(res["result"])
    except RhoovaError as e:
        e.printPretty()
    return result    

In [14]:
# Fix parameters for building yield curve
yieldcurveconfig=        {
            "name": "TRYOIS",
            "settlementDays": 2,
            "intpMethod": "Linear",
            "currency": "TRY",
            "calendar": "Turkey",
            "dayCounter": "ActualActual",
            "period": "1D",
            "index": "TLREF",
            "paymentlag": 1,
            "usedates": False,
            "instruments": {
                "OIS": {
                    "businessDayConvention": "ModifiedFollowing",
                    "dayCounter": "ActualActual",
                    "frequency": "Quarterly"
                }
            }
        }

In [15]:
data = {
    "id": "Portfolio1",
    "name": "Portfolio1",
    "valuationDate": "2025-02-25",
    "valuationCurrency": "TRY",
    "tasks": [
        {
            "trade_id": "Bond_1",
            "calculation_type": "fixed_rate_bond",
            "notional": 1000000,
            "settlementDate": "2025-02-27",
            "buySell": "Buy",
            "currency": "TRY",
            "discountCurve": "TRYOIS",
            "forecastCurve": "TRYOIS",
            "fixedRateBondDefinition": {
                "discountCurve": "TRYOIS",
                "issueDate": "2024-10-09",
                "maturityDate": "2034-09-27",
                "frequency": "Semiannual",
                "coupon": 0.277,
                "calendar": "Turkey",
                "currency": "TRY",
                "dateGeneration": "Backward",
                "dayCounter": "ActualActual",
                "businessDayConvention": "ModifiedFollowing",
                "maturityDateConvention": "ModifiedFollowing",
                "redemption": 100,
                "endOfMonth": True
            }
        }
    ],
    "curves": [
            yieldcurveconfig 
    ],
    "yieldData": yielddata.to_dict("records"),
    "method": "total_present_value",
}



In [18]:
result=get_result(data)["total_present_value"]

In [19]:
bp=1

In [20]:
timebucket=["3M","6M","1Y", "2Y", "3Y", "5Y","10Y","15Y","20Y","30Y"]
keyrisk = []
def portfolio_keyrate_shock(data,timebucket):
    pv=get_result(data)
    keyrate={}
    yieldcurveconfig2=yieldcurveconfig.copy()
    for bucket in timebucket:
        yieldcurveconfig2['applyShock']={"method": "keyrate","shockValues" : [{"tenor":bucket,"shockValue" : bp}]}
        data["curves"]=[yieldcurveconfig2]
        pvplus=get_result(data)
        pvplus[bucket]=pvplus
        keyrate[bucket]=pvplus["total_present_value"]-result
    return keyrate
    

In [21]:
keyratevalues=portfolio_keyrate_shock(data,timebucket)

In [22]:
tenor_map={
"3M":"0.25Y",
"6M":"0.5Y",
"1Y":"1Y",
"2Y":"2Y",
"3Y":"3Y",
"5Y":"5Y",
"10Y":"10Y",
"15Y":"15Y",
"20Y":"20Y",
"30Y":"30Y"}

In [23]:

formula = r"s_{k,r_t} = \frac{V_i(r_t + 0.0001, cs_t) - V_i(r_t, cs_t)}{0.0001}"
display(Math(formula))


<IPython.core.display.Math object>

In [25]:
risk_class = "GIRR"
risk_type = "GIRR_Delta"
qualifier = data["tasks"][0]["fixedRateBondDefinition"]["currency"]
bucket = data["tasks"][0]["fixedRateBondDefinition"]["currency"]
curve_type = data["tasks"][0]["discountCurve"][3:]
curve = data["tasks"][0]["discountCurve"]

sensitivities_df = pd.DataFrame([
    {
        "RiskClass": risk_class,
        "RiskType": risk_type,
        "Qualifier": qualifier,
        "Bucket": bucket,
        "CurveType": curve_type,
        "Curve": curve,
        "Tenor": tenor_map.get(tenor),
        "Sensitivity": sensitivity
    }
    for tenor, sensitivity in keyratevalues.items()
])
sensitivities_df

,RiskClass,RiskType,Qualifier,Bucket,CurveType,Curve,Tenor,Sensitivity
0,GIRR,GIRR_Delta,TRY,TRY,OIS,TRYOIS,0.25Y,0.000000
1,GIRR,GIRR_Delta,TRY,TRY,OIS,TRYOIS,0.5Y,-5.409262
2,GIRR,GIRR_Delta,TRY,TRY,OIS,TRYOIS,1Y,-9.612171
3,GIRR,GIRR_Delta,TRY,TRY,OIS,TRYOIS,2Y,-20.646432
4,GIRR,GIRR_Delta,TRY,TRY,OIS,TRYOIS,3Y,-24.034787
5,GIRR,GIRR_Delta,TRY,TRY,OIS,TRYOIS,5Y,-38.345106
6,GIRR,GIRR_Delta,TRY,TRY,OIS,TRYOIS,10Y,-81.280446
7,GIRR,GIRR_Delta,TRY,TRY,OIS,TRYOIS,15Y,0.000000
8,GIRR,GIRR_Delta,TRY,TRY,OIS,TRYOIS,20Y,0.000000
9,GIRR,GIRR_Delta,TRY,TRY,OIS,TRYOIS,30Y,0.000000


In [27]:
riskweight=pd.read_excel(directory+"/FRTB_Params_BCBS.xlsx")
riskweight=riskweight[["tenor","RiskWeight"]]
riskweight

,tenor,RiskWeight
0,0.25Y,0.017
1,0.5Y,0.017
2,1Y,0.016
3,2Y,0.013
4,3Y,0.012
5,5Y,0.011
6,10Y,0.011
7,15Y,0.011
8,20Y,0.011
9,30Y,0.011


In [28]:
formula = r"WS_k = RW_k s_k"
display(Math(formula))


<IPython.core.display.Math object>

In [29]:
sensitivities = pd.merge(sensitivities_df, riskweight, left_on="Tenor",  right_on="tenor", how="left")
sensitivities["Weighted Sensitivity"]=sensitivities["Sensitivity"]*sensitivities["RiskWeight"]
sensitivities

,RiskClass,RiskType,Qualifier,Bucket,CurveType,Curve,Tenor,Sensitivity,tenor,RiskWeight,Weighted Sensitivity
0,GIRR,GIRR_Delta,TRY,TRY,OIS,TRYOIS,0.25Y,0.000000,0.25Y,0.017,0.000000
1,GIRR,GIRR_Delta,TRY,TRY,OIS,TRYOIS,0.5Y,-5.409262,0.5Y,0.017,-0.091957
2,GIRR,GIRR_Delta,TRY,TRY,OIS,TRYOIS,1Y,-9.612171,1Y,0.016,-0.153795
3,GIRR,GIRR_Delta,TRY,TRY,OIS,TRYOIS,2Y,-20.646432,2Y,0.013,-0.268404
4,GIRR,GIRR_Delta,TRY,TRY,OIS,TRYOIS,3Y,-24.034787,3Y,0.012,-0.288417
5,GIRR,GIRR_Delta,TRY,TRY,OIS,TRYOIS,5Y,-38.345106,5Y,0.011,-0.421796
6,GIRR,GIRR_Delta,TRY,TRY,OIS,TRYOIS,10Y,-81.280446,10Y,0.011,-0.894085
7,GIRR,GIRR_Delta,TRY,TRY,OIS,TRYOIS,15Y,0.000000,15Y,0.011,0.000000
8,GIRR,GIRR_Delta,TRY,TRY,OIS,TRYOIS,20Y,0.000000,20Y,0.011,0.000000
9,GIRR,GIRR_Delta,TRY,TRY,OIS,TRYOIS,30Y,0.000000,30Y,0.011,0.000000


In [30]:

display(Math(r"""
\text{CVR}_k^+ = -\sum_{i} \left[ V_i \left( x_k^{\text{RW(Curvature)}^+} \right) - V_i (x_k) - \text{RW}_k^{\text{Curvature}} \cdot s_{ik} \right]
"""))

display(Math(r"""
\text{CVR}_k^- = -\sum_{i} \left[ V_i \left( x_k^{\text{RW(Curvature)}^-} \right) - V_i (x_k) - \text{RW}_k^{\text{Curvature}} \cdot s_{ik} \right]
"""))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [31]:
#(MAR21, par. 21.99)
riskweightcurvature=riskweight["RiskWeight"].max()
yieldcurveconfig3=yieldcurveconfig.copy()
yieldcurveconfig3['applyShock']={"method": "parallel","shockValues" : [{"tenor":"all","shockValue" : 10000*riskweightcurvature}]}
data["curves"]=[yieldcurveconfig3]
paralel_result_plus=get_result(data)["total_present_value"]

yieldcurveconfig4=yieldcurveconfig.copy()
yieldcurveconfig4['applyShock']={"method": "parallel","shockValues" : [{"tenor":"all","shockValue" : -10000*riskweightcurvature}]}
data["curves"]=[yieldcurveconfig4]
paralel_result_minus=get_result(data)["total_present_value"]

In [32]:
cvrplus=-1*(paralel_result_plus-result-sensitivities["Sensitivity"].sum()*riskweightcurvature)

In [33]:
cvrminus=-1*(paralel_result_minus-result-sensitivities["Sensitivity"].sum()*riskweightcurvature)

In [34]:
curvature_df = [{'RiskClass': 'GIRR', 
         'RiskType': 'GIRR_Curv', 
         'Bucket': 'EUR', 
         'CVR+': cvrplus, 
         'CVR-': cvrminus}]
pd.DataFrame(curvature_df)

,RiskClass,RiskType,Bucket,CVR+,CVR-
0,GIRR,GIRR_Curv,EUR,70722.143476,-79060.277471
